# Elevation Interpolation

In [58]:
# import libraries
import arcpy
from arcpy import env # to set working environmnet
import pandas as pd
import psycopg2
import os # file path managment

### Set up environmnet

In [59]:
# Get CWD

cwd = _dh[0] #this is a universal variable that calls the directory where the .ipynb file is located


# Link it to the QAQC.gdb and make it the workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

# allow files to be overwritten
arcpy.env.overwriteOutput = True 

# set save filepath
savepath = os.path.join(cwd, '..','..','data')

#test
arcpy.env.workspace

'c:\\Users\\MrJDF\\Desktop\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb'

## Query elevation data from database
- use SQL
- may need to convert raster to point data
    - see lab 02
- resample to avoid crashing the computer
--(may already be small enough)

In [5]:
# Query the DB and make a layer
arcpy.management.MakeQueryLayer(r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\34.123.228.238.sde", "elevation_pts", "SELECT * FROM elevation_points", "grid_code", '', "4326", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision', "DEFINE_SPATIAL_PROPERTIES", "DO_NOT_INCLUDE_M_VALUES", "DO_NOT_INCLUDE_Z_VALUES", "0 0 0 0")

<Result 'elevation_pts'>

In [8]:
# Export Query Layer (save to local GDB)
arcpy.conversion.ExportFeatures("elevation_pts", savepath + r"\elevation_pts_ExportFeatures", '', "NOT_USE_ALIAS", 'pointid "pointid" true true false 4 Long 0 10,First,#,elevation_pts,pointid,-1,-1;grid_code "grid_code" true true false 4 Long 0 10,First,#,elevation_pts,grid_code,-1,-1', None)


<Result 'c:\\Users\\MrJDF\\Desktop\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\elevation_pts_ExportFeatures.shp'>

In [60]:
# set variable to local layer file

elevation_pts = savepath + r"\elevation_pts_ExportFeatures"

### Sample the dataset
- define appropriate sample size (with proof)
    - This is done in lab 2 where I downsample the 1m raster to 100m, which uses bilinear interpolation (nearest neighbor), then convert those rasters to point data, selecting the mean value of rasters within the point conversion grid.
    - will need to downsample again for size after interpolation

### Run 3 different interpolation algorithms, save to local gdb
- IDW
- Empirical Bayesian Kriging
- Local Polynomial KDE

In [61]:
# IDW

arcpy.ga.IDW(elevation_pts, "grid_code", "idw_stats", r"idw_lab03", 0.000124525892000065, 2, "NBRTYPE=Standard S_MAJOR=1.76434719213634E-02 S_MINOR=1.76434719213634E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", None)

idw_stats = "idw_stats"

In [62]:
# Empirical Bayesian Kriging

arcpy.ga.EmpiricalBayesianKriging(elevation_pts, "grid_code", "bayesian_kriging_stats", r"bayesian_kriging_lab03", 0.000124525892000065, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=1.76434719213634E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "POWER")

bayesian_kriging_stats = "bayseian_kriging_stats"

bayesian_kriging = r"bayesian_kriging_lab03"

In [63]:
# local polynomial kernel

arcpy.ga.LocalPolynomialInterpolation(elevation_pts, "grid_code", "lcl_polynomial_stats", r"lcl_polynomial_lab03", 0.000124525892000065, 1, "NBRTYPE=Standard S_MAJOR=1.76434719213634E-02 S_MINOR=1.76434719213634E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "EXPONENTIAL", None, "NO_USE_CONDITION_NUMBER", None, None, "PREDICTION")

lcl_poly_stats = "lcl_polynomial_stats"


## Evaluate the interpolated estimates of each model compared to the true DEM
- leave one out CROSS VALIDATION in geoprocessing arcpy
- store this accuracy assessment as a table in postgres
- and in home gdb


In [64]:
# Cross Validation, outputs a point layer with stats including error for each point

# idw
arcpy.ga.CrossValidation("idw_stats", savepath + r"\idw_crossval")

<Result 'c:\\Users\\MrJDF\\Desktop\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\idw_crossval.shp'>

In [65]:
#Bayseian Kriging
arcpy.ga.CrossValidation("Bayesian_Kriging_stats", savepath + r"\bayesian_kriging_crossval")

<Result 'c:\\Users\\MrJDF\\Desktop\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\bayesian_kriging_crossval.shp'>

In [66]:
# local polynomial kernel
arcpy.ga.CrossValidation("lcl_polynomial_stats", savepath + r"\lcl_polynomial_crossval")

<Result 'c:\\Users\\MrJDF\\Desktop\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\lcl_polynomial_crossval.shp'>

In [67]:
# Compare results, output comparison table, table with best fit model by percent accuracy
# outputs a geostatistical comparison in the arc project
arcpy.ga.CompareGeostatisticalLayers("lcl_polynomial_stats;idw_stats;bayesian_kriging_stats", savepath + r"\CompareGeostatisticalLayers1", "BestInterpByComparrison", "SINGLE", "ACCURACY", "ACCURACY PERCENT #", "ACCURACY 1", None)

#TODO visualize this and grab the table!

<Result 'c:\\Users\\MrJDF\\Desktop\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\CompareGeostatisticalLayers1'>


## Find and store differences between ground truth and model predictions in a point layer
- visualize this data in arc online
- save to proj GDB and postgres


### Compute RMSE
- use the stats layers output from earlier

In [68]:
# Empirical Bayesian Kriging RMSE

#create numpy array from featureclass for calculations

input = savepath + r"\bayesian_kriging_crossval.shp"
array = arcpy.da.FeatureClassToNumPyArray(input, ("FID","Error"))

#change  array to df
df = pd.DataFrame(array, columns=["FID", "Error"])

# Calculate squared error, add as new column
df['squared_error'] = df['Error'] ** 2

# Calculate RMSE
krig_rmse = numpy.sqrt(df['squared_error'].mean())

# Display RMSE
print("RMSE: ", krig_rmse)

RMSE:  1.3061990821193148


In [69]:
#IDW RMSE

#create numpy array from featureclass for calculations

input = savepath + r"\idw_crossval.shp"
array = arcpy.da.FeatureClassToNumPyArray(input, ("FID","Error"))

#change  array to df
df = pd.DataFrame(array, columns=["FID", "Error"])

# Calculate squared error, add as new column
df['squared_error'] = df['Error'] ** 2

# Calculate RMSE
idw_rmse = numpy.sqrt(df['squared_error'].mean())

# Display RMSE
print("RMSE: ", idw_rmse)

RMSE:  1.3775090571065738


### Save the interpolated maps to my GDB and PostGIS DB
- convert to point layer and use psycopg2 to make new tables in sql remote db
- save raster to gdb should be basic

In [70]:
# Downsample the best fit interpolated raster layerfor ease of computation and storage

# Set the input raster file path
input_raster = r"bayesian_kriging_lab03"

# Set the output raster file path
output_raster = "elev_bayes_kriging_downsmpl"

# Get the cell size of the input raster
desc = arcpy.Describe(input_raster)
cell_size = desc.meanCellWidth # or desc.meanCellHeight, depending on your analysis

# Set an appropriate cell size for the output raster
output_cell_size = cell_size * 10 # increase the cell size by a factor of 10

# Resample the input raster using bilinear interpolation
arcpy.Resample_management(input_raster, output_raster, output_cell_size, "BILINEAR")


<Result 'c:\\Users\\MrJDF\\Desktop\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\elev_bayes_kriging_downsmpl'>

In [71]:
# convert best fit interpolation raster to point layer

#arcpy.conversion.RasterToPoint("source.img", "c:/output/source.shp", "VALUE")
elevEmpBayesKrg = arcpy.conversion.RasterToPoint("elev_bayes_kriging_downsmpl", savepath + r"\elev_bayesian_kriging", "VALUE")

## Only need to save the best fit layer out, so don't need these
#  
#idw = arcpy.conversion.RasterToPoint("idw_lab03", savepath + r"\idwPts", "VALUE")
#lclPolyKernel = arcpy.conversion.RasterToPoint("lcl_polynomial_lab03", savepath + r"\lclPolyPts", "VALUE")


In [72]:
#Local Polynomial Kernel RMSE

#create numpy array from featureclass for calculations

input = savepath + r"\lcl_polynomial_crossval.shp"
array = arcpy.da.FeatureClassToNumPyArray(input, ("FID","Error"))

#change  array to df
df = pd.DataFrame(array, columns=["FID", "Error"])

# Calculate squared error, add as new column
df['squared_error'] = df['Error'] ** 2

# Calculate RMSE
lp_rmse = numpy.sqrt(df['squared_error'].mean())

# Display RMSE
print("RMSE: ", lp_rmse)

RMSE:  1.2748387583349785


In [73]:
#bring the RMSE values together in a pd df to compare

elev_compare = pd.DataFrame({'method':['kriging', 'idw', 'local_polynomial'],
                            'RMSE':[krig_rmse, idw_rmse, lp_rmse]})
elev_compare


,method,RMSE
0,kriging,1.306199
1,idw,1.377509
2,local_polynomial,1.274839


In [74]:
# Get index of row with lowest value of rmse
min_index = elev_compare['RMSE'].idxmin()

# Get method with lowest rmse
min_id = elev_compare.loc[min_index, 'method']

print("Method with the lowest RMSE value is", min_id)

Method with the lowest RMSE value is local_polynomial


## save data to postgres

In [75]:
# Get credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].split(', ')

# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

try:
    conn = psycopg2.connect(**pg_connection_dict)
    print("connected")
except:
    print("connection failed")

connected


In [76]:
# Create and fill kriging results table in postgres db

points = os.path.join(savepath, "elev_bayesian_kriging.shp")
fields_points = ['pointid', 'grid_code', "SHAPE@WKT"]

# Create SQL table
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS elev_kriging")
cursor.execute("""
    CREATE TABLE elev_kriging (
        id SERIAL,
        pointid INT,
        grid_code FLOAT,
        geometry geometry
        )
""")
conn.commit()

print("Table created.")

# Populate PostGIS
with arcpy.da.SearchCursor(points, fields_points) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute("INSERT INTO elev_kriging (pointid, grid_code, geometry) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))
        conn.commit()


print("Operation complete.")


Table created.
Operation complete.


In [77]:
# Create and fill kriging error table in postgres db

points = os.path.join(savepath, "bayesian_kriging_crossval.shp")
fields_points = ['FID','Error', "SHAPE@WKT"]

# Create SQL table
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS elev_kriging_errors")
cursor.execute("""
    CREATE TABLE elev_kriging_errors (
        id SERIAL,
        fid INT,
        error DOUBLE PRECISION,
        geometry geometry
        )
""")
conn.commit()

print("Table created.")

# Populate PostGIS
with arcpy.da.SearchCursor(points, fields_points) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute("INSERT INTO elev_kriging_errors (fid, error, geometry) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))
        conn.commit()

print("Operation complete. Please close connection by running the next cell.")

Table created.
Operation complete. Please close connection by running the next cell.


In [78]:
conn.close

<function connection.close>